### Resources
- [Algorithms for Competitive Programming](https://cp-algorithms.com)
- [Template](#Template_)

### Implementations

- [Binary search](#Binary_search_)
- [Binary_indexed_tree](#Binary_indexed_tree_)
- [Prefix function](#Prefix_function_)
- [Z function](#Z_function_)
- [Prime decomposition](#Prime_decomposition_)
- [Consecutive less/greater](#Consecutive_less/greater_)
- [Disjoint set union](#Disjoint_set_union_)
- [Graph traversal](#Graph_traversal_)
- [Trie](#Trie_)
- [Longest increasing subsequence](#Longest_increasing_subsequence_)
- [Gray code](#Gray_code_)
- Todo: Meet in the middle: split problem in two, process each half, and merge. Note that it is not a recursive approach. It usually drops complexity from $2^n$ to $n2^{n/2}$.
- Todo:  Square root decomposition: to process some range queries, $\sqrt{n}$ complexity.

### Template <span id=Template_></span>

In [ ]:
import collections
import contextlib
import dataclasses
import itertools
import functools
import sys
import random
import time

DEBUG = True


@contextlib.contextmanager
def time_it(name: str = "", file=sys.stderr):
    begin = time.perf_counter()
    yield
    end = time.perf_counter()
    name = name + ":\t" if name else ""
    delta_ms = 1000 * (end - begin)
    if DEBUG:
        print(f"{name}{delta_ms:.3f} ms", file=file)

@dataclasses.dataclass
class Input:
    ...
    
@dataclasses.dataclass
class Output:
    ...
    

def solve(input_: Input) -> Output:
    ...
    

def generate_test_case(seed: int) -> Input:
    ...

def brute_force_solve(input_: Input) -> Output:
    ...

class WrongAnswerException(Exception):
    ...

def test():
    global DEBUG
    DEBUG = False
    
    # Base test cases
    assert solve(...) == ...
    
    # Random test cases
    for seed in range(100):
        input_ = generate_test_case(seed=seed)
        expected = brute_force_solve(input_)
        received = solve(input_)
        if expected != received:
            raise WrongAnswerException(
                f"Wrong answer\nInput:\t{input_}\nExpected:\t{expected}\nReceived:\t{received}"
            )
    
    DEBUG = False
    # Stress testing
    for _ in range(10):
        generate_test_case(seed=float("inf"))
    
    print("Ok")

test()
stress_test()

### Binary search <span id=Binary_search_></span>

In [21]:
def argsort(values):
    return sorted(range(len(values)), key=values.__getitem__)

def binsearch(low, high, do_search_less_than):
    while high - low > 1:
        mid = (low + high) // 2
        if do_search_less_than(mid):
            high = mid
        else:
            low = mid
    return low

### Binary indexed tree <span id=Binary_indexed_tree_></span>

In [1]:
def leave_only_least_significant_bit(x):
    return x & ~(x - 1)

class BinaryIndexedTree:
    """Aka Fenwick tree."""
    def __init__(self, size):
        self.size = size
        self.tree = [0] * (size + 1)
        
    def update(self, index, value):
        index += 1
        while index <= self.size:
            self.tree[index] += value
            index += leave_only_least_significant_bit(index)
            
    def get_prefix_sum(self, index):
        index += 1
        prefix_sum = 0
        while index:
            prefix_sum += self.tree[index]
            index -= leave_only_least_significant_bit(index)
        return prefix_sum

    def get_range_sum(self, begin, end):
        return self.get_prefix_sum(end - 1) - self.get_prefix_sum(begin - 1)

### Prefix function <span id=Prefix_function_></span>

In [4]:
def compute_prefix_function(string):
    prefix_function = []
    next_border = lambda i: prefix_function[i - 1] if i > 0 else None 
    for i, letter in enumerate(string):
        i = next_border(i)
        while i is not None and string[i] != letter:
            i = next_border(i)
            
        prefix_function.append(i + 1 if i is not None else 0)

    return prefix_function

def find_word(string, word):
    s = len(string)
    w = len(word)
    prefix = compute_prefix_function(word + '\0' + string)
    indices = [i for i in range(s - w + 1) if prefix[2 * w + i] == w]
    return indices

### Z function <span id=Z_function_></span>

In [2]:
def compute_z_function(string):
    n = len(string)
    z_function = [0] * n
    z_function[0] = n
    argmax_i_plus_z_i = 1
    for i in range(1, n):
        index_minus_argmax = i - argmax_i_plus_z_i
        new_max_z_value = max(0, z_function[argmax_i_plus_z_i] - index_minus_argmax)
        
        if z_function[index_minus_argmax] < new_max_z_value:
            z_function[i] = z_function[index_minus_argmax]
            continue

        while i + new_max_z_value < n and string[new_max_z_value] == string[i + new_max_z_value]:
            new_max_z_value += 1
            
        argmax_i_plus_z_i = i
        z_function[i] = new_max_z_value
    
    return z_function

### Prime decomposition <span id=Prime_decomposition_></span>

In [88]:
def find_max_prime_divisor_for_nums(max_num):
    prime_divisors = [None] * (max_num + 1)  
    for prime in range(2, max_num + 1):
        if prime_divisors[prime] is None:
            for i in range(prime, max_num + 1, prime):
                prime_divisors[i] = prime
            
    return prime_divisors

### Consecutive less/greater <span id=Consecutive_less/greater_></span>

In [102]:
import bisect

def count_consecutive_less_to_the_left(nums):
    sorted_prefix_subsequence = [(float("inf"), -1)]
    result = []
    for i, num in enumerate(nums):
        while sorted_prefix_subsequence[-1][0] < num:
            sorted_prefix_subsequence.pop()
        result.append(i - sorted_prefix_subsequence[-1][1] - 1)
        sorted_prefix_subsequence.append((num, i))
        
    return result

def count_consecutive_not_greater_to_the_right(nums):
    sorted_suffix_subsequence = [(float("inf"), len(nums))]
    result = []
    for i in reversed(range(len(nums))):
        num = nums[i]
        while sorted_suffix_subsequence[-1][0] <= num:
            sorted_suffix_subsequence.pop()
        result.append(sorted_suffix_subsequence[-1][1] - i - 1)
        sorted_suffix_subsequence.append((num, i))
        
    return result[::-1]

### Disjoint set union <span id=Disjoint_set_union_></span>

In [2]:
class DisjointSets:
    def __init__(self, size):
        self.parents = list(range(size))
        self.sizes = [1] * size
    
    def find(self, node):
        if self.parents[node] != node:
            self.parents[node] = self.find(self.parents[node])
        return self.parents[node]

    def union(self, first, second):
        first = self.find(first)
        second = self.find(second)
        if first == second:
            return
        if self.sizes[first] > self.sizes[second]:
            first, second = second, first
        self.parents[first] = second
        self.sizes[second] += self.sizes[first]

### Graph traversal <span id=Graph_traversal_></span>

In [ ]:
import abc
import collections
import dataclasses
import typing
import queue


class Traversal(abc.ABC):
    @abc.abstractmethod
    def NodesAdjacentTo(self, node) -> typing.Iterable:
        pass
    
    def OnNodeEnter(self, node) -> None:
        pass

    def OnNodeExit(self, node) -> None:
        pass

def depth_first_search(traversal: Traversal, node):
    traversal.OnNodeEnter(node)
    
    for adjacent_node in traversal.NodesAdjacentTo(node):
        depth_first_search(traversal, adjacent_node)

    traversal.OnNodeExit(node)
    
def breadth_first_search(traversal: Traversal, queue):
    queue = collections.deque(queue)

    while queue:
        node = queue.popleft()
        traversal.OnNodeEnter(node)

        for adjacent_node in traversal.NodesAdjacentTo(node):
            queue.append(adjacent_node)

        traversal.OnNodeExit(node)
        

### Trie <span id=Trie_></span>

In [18]:
class Trie:
    def __init__(self):
        self.transitions = {}
        self.is_terminal = False
        
    def add(self, word):
        node = self
        for letter in word:
            if letter not in node.transitions:
                node.transitions[letter] = Trie()
            node = node.transitions[letter]
        node.is_terminal = True

### Longest increasing subsequence <span id=Longest_increasing_subsequence_></span>

In [134]:
import bisect

def len_of_longest_increasing_subsequence(nums) -> int:
    subsequence = []
    # Invariant: subsequence[i] is minimal last element for increasing subsequence of size i + 1
    for num in nums:
        index = bisect.bisect_left(subsequence, num)
        if index == len(subsequence):
            subsequence.append(num)
        else:
            subsequence[index] = num
            
    return len(subsequence)

### Gray code <span id=Gray_code_></span>

In [135]:
def get_gray_code(n):
    return n ^ (n >> 1)

def next_gray_code_delta(n):
    gray_code = get_gray_code(n)
    next_gray_code = get_gray_code(n + 1)
    sign = -1 if next_gray_code < gray_code else 1
    bit = abs(next_gray_code - gray_code).bit_length()
    return sign * bit

# print(f"{get_gray_code(3):0{3}b}")

010
